In [1]:
from weiner_variation.sim.data_structures import DrawDurations
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, create_in_profile, REVERSING_PAUSE_DURATION, DIAMETER_STD, TEMPERATURE_STD
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm, weibull_min
from copy import deepcopy
import tqdm
from multiprocessing import Pool

/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (depth - l23(_alpha) * np.sin(_alpha) - fh(_alpha)) / (1 - np.cos(_alpha))
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:83: RuntimeWarning: invalid value encountered in scalar multiply
  width / 2 - _r2 * np.sin(_alpha) - l23(_alpha) * np.cos(_alpha)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  - fw(_alpha) - (_r2 + r4) * np.sin(_alpha4)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (de

In [2]:
import pyroll.basic as pr

In [3]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = "sim_durations_results.csv"

In [4]:
df_durations = pd.read_csv(DATA_DIR / "duo_pauses_dist.csv", header=0, index_col=0)
df_durations

,mean,std,max,min,loc,alpha,beta
R1-R2,6.633088,1.471388,11.047252,5.115,5.425000,1.248549,1.070947
R2-R3,4.435441,0.995052,7.420597,3.040,3.002936,6.802674,0.198825
R3-R4,5.988015,2.192780,12.566355,4.015,4.507276,1.823817,0.733746
R4-R5,5.654853,0.470526,7.066431,4.745,4.641153,4.547398,0.232675
R5-R6,5.044559,0.566902,6.745266,3.635,3.635000,6.605120,0.231032
R6-R7,5.195588,0.588969,6.962496,4.235,4.235579,6.937555,0.125272
R7-R8,6.435882,1.557230,11.107573,4.705,5.160270,1.637811,0.746130
R8-R9,6.080735,0.723092,8.250011,4.745,4.724997,3.723512,0.323734
R9-R10,7.008235,0.775194,9.333816,5.260,5.271222,6.443636,0.286210
R10-F1,9.233309,0.399014,10.430352,8.290,8.260580,12.416743,0.074943


In [11]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)
durations_dists = [weibull_min(c=r["shape"], scale=r["scale"]) for i, r in df_durations.iterrows()]

In [17]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
durations = np.concatenate([
    d.rvs(random_state=RNG, size=(SAMPLE_COUNT, 1))
    for d in durations_dists[:-1]
], axis=1)

draws = [
    DrawDurations(d, t, dur)
    for d, t, dur in zip(diameters, temperatures, durations)
]

In [19]:
def worker(draw: DrawDurations):    
    ip = create_in_profile(
            diameter=draw.diameter,
            temperature=draw.temperature,
    )

    sequence = deepcopy(PASS_SEQUENCE)  
    
    transports = [u for u in sequence if isinstance(u, pr.Transport)]
    for t, d in zip(transports, draw.durations):
        t.duration = d
    
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

In [20]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:23<00:00, 42.10it/s]


In [21]:
df = results |> filter$(e -> "error" not in e) |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

draw                                                                  \
     diameter  temperature                                          durations   
0    0.050390  1429.101737  [6.225537677899436, 4.54296501256564, 5.265606...   
1    0.050772  1419.874552  [8.187268054976624, 4.70239621130067, 5.689990...   
2    0.051325  1420.567525  [6.801212818430111, 4.599137474997898, 6.68950...   
3    0.049996  1424.015852  [7.778068149851444, 4.622829810887898, 5.47283...   
4    0.050787  1434.180538  [7.949950776287229, 3.5323327002212577, 5.7709...   
..        ...          ...                                                ...   
995  0.050207  1427.757346  [5.517864478318161, 4.295890828561809, 4.70162...   
996  0.049552  1424.855232  [7.243299613232511, 4.556098058672385, 4.98895...   
997  0.047715  1433.136323  [6.584720615517212, 3.5555449730944866, 5.7855...   
998  0.049303  1418.515979  [5.930809359359286, 4.27418711021144, 5.748456...   
999  0.048064  1433.311226  [5.726185113390319, 4.5419116196466085, 6.2723...   

        roll_force                                               \
                R1             R2             R3             R4   
0    394196.462853  228547.814362  417117.203989  176585.885513   
1    410272.858329  239727.021655  431551.766297  182154.315166   
2    421243.534507  241557.665076  428966.407126  182314.190028   
3    390485.057033  230300.184001  423804.406655  180146.906581   
4    397863.504123  232908.103537  416924.177591  176895.862468   
..             ...            ...            ...            ...   
995  391588.249875  226252.150674  414383.972788  174728.214465   
996  380647.338849  225020.375144  419492.034977  178325.075103   
997  337404.190328  204190.087582  400134.262157  175848.495484   
998  380926.049933  223273.178544  418164.064309  179770.171138   
999  344110.896238  205744.244649  402372.812233  177044.580846   

                                                  ...    strain            \
                R5             R6             R7  ...        R5        R6   
0    250736.582493  133931.153519  250350.455282  ...  0.596736  0.499441   
1    256891.424491  137851.376688  256782.677129  ...  0.596886  0.499530   
2    256574.793568  137402.983322  254355.116180  ...  0.597011  0.499566   
3    254234.050413  135366.817773  254381.365889  ...  0.596661  0.499440   
4    251190.810799  134878.583507  252013.592846  ...  0.596838  0.499478   
..             ...            ...            ...  ...       ...       ...   
995  248082.857246  133530.829865  250135.480260  ...  0.596672  0.499407   
996  252757.553120  133712.758261  250405.450433  ...  0.596519  0.499383   
997  248902.075005  132885.871836  248518.114164  ...  0.595846  0.499119   
998  253957.324688  137251.892196  255779.810705  ...  0.596451  0.499378   
999  254116.900764  135402.383488  252825.948889  ...  0.596005  0.499201   

                                                                           \
           R7        R8        R9       R10        F1        F2        F3   
0    0.728155  0.596423  0.458517  0.459896  0.342764  0.346804  0.447988   
1    0.728209  0.596499  0.458562  0.459911  0.342747  0.346715  0.447944   
2    0.728214  0.596473  0.458524  0.459824  0.342710  0.346700  0.447941   
3    0.728168  0.596447  0.458502  0.459800  0.342685  0.346645  0.447912   
4    0.728173  0.596428  0.458478  0.459751  0.342677  0.346664  0.447925   
..        ...       ...       ...       ...       ...       ...       ...   
995  0.728142  0.596419  0.458509  0.459848  0.342733  0.346735  0.447958   
996  0.728136  0.596420  0.458505  0.459825  0.342714  0.346699  0.447940   
997  0.728044  0.596361  0.458454  0.459752  0.342680  0.346669  0.447928   
998  0.728156  0.596458  0.458509  0.459793  0.342677  0.346637  0.447909   
999  0.728087  0.596407  0.458469  0.459726  0.342645  0.346612  0.447900   

               
           F4  
0    0.416677  
1    0.416589  
2    0.416588  

In [10]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)